# Trabajo final -- formulario de carga de datos y respuestas

En este notebook pueden ingresar sus respuestas a las preguntas del trabajo final y asegurarse que sus comandos SQL funcionan de manera adecuada.

**IMPORTANTE**: **Debes entregar tu notebook ejecutado según tus últimos cambios ya que se revisarán los resultados en base a lo que salga ejecutado del notebook.**

En el menú superior de google colab > Entorno de ejecución > Ejecutar todo,  tienes la opción de ejecutar todas las celdas del notebook. Puedes realizar esta acción antes de entregar tu tarea para asegurarte de que tus últimos cambios en el código sean los ejecutados.




### Cargar SQLite y subir la base de datos a Google Colab

Cómo siempre, para poder ocupar el notebook, lo deben subir a https://colab.research.google.com

Luego, ejecutando el siguiente campo, se crea una instancia de servidor SQLite, se creará el esquema vista en el enunciado, y se cargarán los datos desde un google drive.

In [112]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# pin package versions for google colab compatibility
!pip install SQLAlchemy==1.4.46
!pip install ipython-sql==0.4.1
%load_ext sql

# Cargamos SQL y la base de datos:
!wget -O datos_T3.db --no-check-certificate https://drive.google.com/uc\?export\=download\&id\=14puII6h6V45QY_FwRsgY_g86y7hf9IcE

%sql sqlite:///datos_T3.db

# Descargamos los csv:
!wget -O Ciudad.csv --no-check-certificate https://drive.google.com/uc\?export\=download\&id\=1ntPogacmnWcpePtYA6vX_ZCb30873wtL
!wget -O Pais.csv --no-check-certificate https://drive.google.com/uc\?export\=download\&id\=1MWHx372JwSYyWkD7Jtv4WJNDBOvdrqSg
!wget -O CiudadPais.csv --no-check-certificate https://drive.google.com/uc\?export\=download\&id\=1aDxin4Y1cgZ43jVcIC_6_mz4_FdFoWmN

# Importamos Pandas:
import pandas as pd


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/bin/bash: service: command not found
sudo: unknown user: postgres
sudo: error initializing audit plugin sudoers_audit
You should consider upgrading via the '/Users/ldavico/.pyenv/versions/3.9.12/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ldavico/.pyenv/versions/3.9.12/bin/python3.9 -m pip install --upgrade pip' command.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
--2023-05-13 17:24:11--  https://drive.google.com/uc?export=download&id=14puII6h6V45QY_FwRsgY_g86y7hf9IcE
Resolviendo drive.google.com (drive.google.com)... 142.250.0.139, 142.250.0.101, 142.250.0.113, ...
Conectando con drive.google.com (drive.google.com)[142.250.0.139]:443... conectado.
Petición HTTP enviada, esperando respuesta... 303 See Other
Localización: https://doc-08-88-docs.googleusercontent.co

## Pregunta 1

En esta primera pregunta, te pedimos ejecutar la primera celda del  notebook para configurar el entorno de SQL, y descargar la base de datos de tareas 2/3  además de los tres archivos CSV.

Luego, te pedimos crear las tablas nuevas con los atributos adecuados (los nombres deben coincidir con los nombres de la especificación de arriba, y los tipos de atributos los debes asignar tú). Para guiar un poco tu diseño de distintos tipos de atributos, te dejamos dos ejemplos con tipos de datos especificados:

```
Persona(rut VARCHAR(100) PRIMARY KEY, nombre VARCHAR(100), correo VARCHAR(100), telefono VARCHAR(100));

Paises(pid INT PRIMARY KEY, nombre VARCHAR(100));
```

Para esto, debes escribir un comando del tipo `CREATE TABLE…` en SQL para cada tabla marcada con *** en la especificación de arriba. El punto del ejercicio es modificar tu base de datos tal que al final de la tarea contenga los datos necesarios. Quiere decir que, algunas tablas se mantienen como están (por ejemplo Persona) y algunas se pueden borrar al final del ejercicio.

**Importante**: No te olvides de especificar las llaves primarias y foráneas. Por ejemplo, tenemos que IDCiudad es una llave foránea en la tabla LugarNuevo (para Ciudades), o que rut es la llave foránea en Participa.



### Respuesta

Instanciamos el cursor

In [113]:
# Creamos el cursor
import sqlite3

conn = sqlite3.connect( 'datos_T3.db'  )
conn.text_factory = lambda x: str(x, 'utf-8' )
curr = conn.cursor( )


Creamos las tablas

In [114]:
# Taller
curr.execute('''
        CREATE TABLE IF NOT EXISTS Taller(
        id INTEGER NOT NULL PRIMARY KEY,
        nombreTaller VARCHAR(100),
        nombreProfesor VARCHAR(100),
        fecha DATE,
        hora TIME,
        valor INTEGER,
        duracion INTEGER
        )
    ''')

# Deportivo
curr.execute('''
        CREATE TABLE IF NOT EXISTS Deportivo(
        id INTEGER NOT NULL PRIMARY KEY,
        deporte VARCHAR(100),
        nrEquipos INTEGER,
        nrPersonas INTEGER,
        FOREIGN KEY (id) REFERENCES Taller(id)
        )
    ''')

# Instrumental
curr.execute('''
        CREATE TABLE IF NOT EXISTS Instrumental(
        id INTEGER NOT NULL PRIMARY KEY,
        instrumento VARCHAR(100),
        nrInstrumentos INTEGER,
        FOREIGN KEY (id) REFERENCES Taller(id)
        )
    ''')

# Pais
curr.execute('''
        CREATE TABLE IF NOT EXISTS Pais(
        id INTEGER NOT NULL PRIMARY KEY,
        nombrePais VARCHAR(100)
        )
    ''')

# Ciudad
curr.execute('''
        CREATE TABLE IF NOT EXISTS Ciudad(
        id INTEGER NOT NULL PRIMARY KEY,
        NombreCiudad VARCHAR(100),
        idPais INTEGER NOT NULL,
        FOREIGN KEY (idPais) REFERENCES Pais(id)
        )
    ''')

# LugarNuevo
curr.execute('''
        CREATE TABLE IF NOT EXISTS LugarNuevo(
        id INTEGER NOT NULL PRIMARY KEY,
        calle VARCHAR(100),
        numero INTEGER,
        codigo VARCHAR(100),
        IDCiudad INTEGER NOT NULL,
        FOREIGN KEY (IDCiudad) REFERENCES Ciudad(id)
        )
    ''')


Chequeamos que estén correctos los esquemas recién creados

In [115]:
# Taller
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'Taller';
''')

schema = curr.fetchall()
print(schema[0][0])

# Deportivo
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'Deportivo';
''')

schema = curr.fetchall()
print(schema[0][0])

# Instrumental
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'Instrumental';
''')

schema = curr.fetchall()
print(schema[0][0])

# Pais
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'Pais';
''')

schema = curr.fetchall()
print(schema[0][0])

# Ciudad
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'Ciudad';
''')

schema = curr.fetchall()
print(schema[0][0])

# LugarNuevo
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'LugarNuevo';
''')

schema = curr.fetchall()
print(schema[0][0])

CREATE TABLE Taller(
        id INTEGER NOT NULL PRIMARY KEY,
        nombreTaller VARCHAR(100),
        nombreProfesor VARCHAR(100),
        fecha DATE,
        hora TIME,
        valor INTEGER,
        duracion INTEGER
        )
CREATE TABLE Deportivo(
        id INTEGER NOT NULL PRIMARY KEY,
        deporte VARCHAR(100),
        nrEquipos INTEGER,
        nrPersonas INTEGER,
        FOREIGN KEY (id) REFERENCES Taller(id)
        )
CREATE TABLE Instrumental(
        id INTEGER NOT NULL PRIMARY KEY,
        instrumento VARCHAR(100),
        nrInstrumentos INTEGER,
        FOREIGN KEY (id) REFERENCES Taller(id)
        )
CREATE TABLE Pais(
        id INTEGER NOT NULL PRIMARY KEY,
        nombrePais VARCHAR(100)
        )
CREATE TABLE Ciudad(
        id INTEGER NOT NULL PRIMARY KEY,
        NombreCiudad VARCHAR(100),
        idPais INTEGER NOT NULL,
        FOREIGN KEY (idPais) REFERENCES Pais(id)
        )
CREATE TABLE LugarNuevo(
        id INTEGER NOT NULL PRIMARY KEY,
        calle 

Chequeamos los esquemas de las otras tablas para revisar que todo esté en orden

In [116]:
# Persona
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'Persona';
''')

persona_schema = curr.fetchall()
print(persona_schema[0][0])

# Participa
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'Participa';
''')

participa_schema = curr.fetchall()
print(participa_schema[0][0])

# EstaEn
curr.execute('''
    SELECT sql 
    FROM sqlite_schema 
    WHERE tbl_name = 'EstaEn';
''')

estaen_schema = curr.fetchall()
print(estaen_schema[0][0])

CREATE TABLE Persona(nombre VARCHAR(100), rut VARCHAR(100) PRIMARY KEY, telefono VARCHAR(100), correo VARCHAR(100))
CREATE TABLE Participa(rut VARCHAR(100), idTaller INT, FOREIGN KEY (rut) REFERENCES Persona(rut), PRIMARY KEY (rut, idTaller))
CREATE TABLE EstaEn(idTaller INT PRIMARY KEY, idLugar INT, FOREIGN KEY (idLugar) REFERENCES Lugar(idLugar))


## Pregunta 2

Inserta a las tablas `Taller`, `Deportivo` e `Instumental` los datos que se piden según su esquema, desde las tablas `TallerDeportivo` y `TallerInstrumental` que ya existen en tu base de datos.

Para esto puedes ocupar SQL, Pandas, Python, lo que sea. Una buena alternativa se propone en el siguiente link: https://www.w3schools.com/sql/sql_insert_into_select.asp

También pueden ver el siguiente link: https://stackoverflow.com/questions/71362727/python-sqlite3-insert-data-from-for-loop (si siguen esta forma, recuerden hacer commit a la connection luego de ejecutar el INSERT INTO )

### Respuesta

Primero veremos lo que tienen las tablas **TallerDeportivo** y **TallerInstrumental**

In [117]:
# TallerDeportivo
curr.execute('''
    SELECT * 
    FROM TallerDeportivo
''')

tallerDeportivo = pd.DataFrame(curr.fetchall(), columns=["id","nombreTaller", "nombreProfesor", 
"fecha","hora","valor","duracion","deporte","nrEquipos", "nrPersonas"])
tallerDeportivo.head()

,id,nombreTaller,nombreProfesor,fecha,hora,valor,duracion,deporte,nrEquipos,nrPersonas
0,1,Taller deportivo 1,Felicia Rivera,2023-03-24,17:00,8,1.0,Handbol,2,7
1,2,Taller deportivo 2,Tyler Ferguson,2023-05-09,11:00,8,1.5,Handbol,2,7
2,3,Taller deportivo 3,Chris Bryant,2023-01-16,15:00,6,2.0,Voleibol,2,6
3,4,Taller deportivo 4,Gregg Thompson,2023-03-22,13:00,8,2.0,Handbol,2,7
4,5,Taller deportivo 5,Joanna Washington,2022-06-30,13:00,2,2.0,Voleibol,2,6


In [118]:
# TallerInstrumental
curr.execute('''
    SELECT * 
    FROM TallerInstrumental
''')

tallerInstrumental = pd.DataFrame(curr.fetchall(), columns=["id","nombreTaller", "nombreProfesor", 
"fecha","hora","valor","duracion","instrumento","nrInstrumentos"])
tallerInstrumental.head()

,id,nombreTaller,nombreProfesor,fecha,hora,valor,duracion,instrumento,nrInstrumentos
0,101,Taller Instrumental 1,Mr. Michael Mcguire,2022-11-03,11:00,4,1.0,Trompeta,12
1,102,Taller Instrumental 2,Cameron Evans,2022-01-04,15:00,3,1.5,Bajo,2
2,103,Taller Instrumental 3,Sheila Lewis,2022-04-04,13:00,8,1.5,Guitarra,4
3,104,Taller Instrumental 4,Ebony Smith,2022-10-25,14:00,3,1.0,Guitarra,4
4,105,Taller Instrumental 5,Jose Powell,2022-05-09,17:00,5,1.5,Bajo,21


Ahora, procedemos a cargar los datos primero en **Taller** y luego en **Deportivo** e **Instrumental**.

In [119]:
cols_taller = ["id", "nombreTaller", "nombreProfesor", "fecha", "hora", "valor", "duracion"]
df_deportivo = tallerDeportivo[cols_taller]
df_instrumental = tallerInstrumental[cols_taller]
df_instrumental.head()

# Creamos un dataframe unico de taller
df_taller = df_deportivo.merge(df_instrumental, how='outer')
df_taller.head(200)

,id,nombreTaller,nombreProfesor,fecha,hora,valor,duracion
0,1,Taller deportivo 1,Felicia Rivera,2023-03-24,17:00,8,1.0
1,2,Taller deportivo 2,Tyler Ferguson,2023-05-09,11:00,8,1.5
2,3,Taller deportivo 3,Chris Bryant,2023-01-16,15:00,6,2.0
3,4,Taller deportivo 4,Gregg Thompson,2023-03-22,13:00,8,2.0
4,5,Taller deportivo 5,Joanna Washington,2022-06-30,13:00,2,2.0
...,...,...,...,...,...,...,...
195,196,Taller Instrumental 96,Matthew Bates,2022-05-23,15:00,2,1.5
196,197,Taller Instrumental 97,Belinda Harper,2023-03-04,14:00,4,2.0
197,198,Taller Instrumental 98,Kerry Stevenson,2022-08-09,17:00,3,2.0
198,199,Taller Instrumental 99,Derrick Lee,2021-10-08,17:00,10,1.0


Llenamos la tabla **Taller** con la información del dataframe. **SOLO EJECUTAR UNA VEZ**, ya que el id debe ser unico y si ya existe se violará la restricción y el código arrojará una excepción. Se puede modificar el atributo *if_exists* con el valor *'replace'* si se quiere evitar este comportamiento.

In [120]:
df_taller.to_sql(
    "Taller",
    con=conn,
    if_exists='append',
    index=False
)

200

Ahora llenamos las tablas **Deportivo** e **Instrumental**

In [121]:
# Deportivo
cols_deportivo = ["id", "deporte", "nrEquipos", "nrPersonas"]
df_deportivo_short = tallerDeportivo[cols_deportivo]

# Instrumental
cols_instrumental = ["id", "instrumento", "nrInstrumentos"]
df_instrumental_short = tallerInstrumental[cols_instrumental]

print(df_deportivo_short.head())
print(df_instrumental_short.head())

   id   deporte  nrEquipos nrPersonas
0   1   Handbol          2          7
1   2   Handbol          2          7
2   3  Voleibol          2          6
3   4   Handbol          2          7
4   5  Voleibol          2          6
    id instrumento  nrInstrumentos
0  101    Trompeta              12
1  102        Bajo               2
2  103    Guitarra               4
3  104    Guitarra               4
4  105        Bajo              21


In [122]:
df_deportivo_short.to_sql(
    "Deportivo",
    con=conn,
    if_exists='append',
    index=False
)

100

In [123]:
df_instrumental_short.to_sql(
    "Instrumental",
    con=conn,
    if_exists='append',
    index=False
)

100

Finalmente, ejecuta las siguientes celdas para corroborar que los datos se hayan cargado de forma correcta:

In [124]:
%%sql

SELECT * FROM Taller LIMIT 5

 * sqlite:///datos_T3.db
Done.


id,nombreTaller,nombreProfesor,fecha,hora,valor,duracion
1,Taller deportivo 1,Felicia Rivera,2023-03-24,17:00,8,1
2,Taller deportivo 2,Tyler Ferguson,2023-05-09,11:00,8,1.5
3,Taller deportivo 3,Chris Bryant,2023-01-16,15:00,6,2
4,Taller deportivo 4,Gregg Thompson,2023-03-22,13:00,8,2
5,Taller deportivo 5,Joanna Washington,2022-06-30,13:00,2,2


In [125]:
%%sql
SELECT * FROM Deportivo LIMIT 10

 * sqlite:///datos_T3.db
Done.


id,deporte,nrEquipos,nrPersonas
1,Handbol,2,7
2,Handbol,2,7
3,Voleibol,2,6
4,Handbol,2,7
5,Voleibol,2,6
6,Fútbol,2,11
7,Zumba,1,
8,Zumba,1,
9,Handbol,2,7
10,Zumba,1,


In [126]:
%%sql
SELECT * FROM Instrumental LIMIT 10

 * sqlite:///datos_T3.db
Done.


id,instrumento,nrInstrumentos
101,Trompeta,12
102,Bajo,2
103,Guitarra,4
104,Guitarra,4
105,Bajo,21
106,Violín,10
107,Trompeta,4
108,Batería,10
109,Piano,5
110,Bajo,9


## Pregunta 3

Ocupando los 3 csv entregados, llena las tablas `País` y `Ciudad` del esquema nuevo. Para esto, hay que hacer un join entre los csv. De nuevo, esto lo puedes hacer con Pandas (cargando los tres CSV a SQL)  o con Python.

Aquí puedes cargar los csvs en Pandas, y luego ocupar este tutorial para cargar datos desde un dataframe a una tabla SQL: https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html

Alternativamente, lo más fácil sería cargar datos desde un csv directamente a sqlite, como se explica aquí: https://www.sqlitetutorial.net/sqlite-import-csv/
 (pero para esto, debes tener sqlite3 instalado en tu computador)

Otro link que puede ser de utilidad es el siguiente: https://kontext.tech/article/633/pandas-save-dataframe-to-sqlite

### Respuesta

Cargamos a DataFrame los 3 csvs. En el caso del Dataframe *df_ciudad*, este ya contendrá los datos correctos.

In [127]:
raw_df_ciudad = pd.read_csv("Ciudad.csv", sep=";")

In [128]:
df_pais = pd.read_csv("Pais.csv", sep=";")

In [129]:
df_ciudad_pais = pd.read_csv("CiudadPais.csv", sep=";")

Hacemos un join entre los dataframe *df_ciudad* y *df_ciudad_pais* para asignar el ID del pais a la ciudad en cuestion

In [130]:
df_ciudad = raw_df_ciudad.merge(df_ciudad_pais, how='inner', on="idCiudad") 
df_ciudad.head()

,idCiudad,nombreCiudad,idPais
0,1,Santiago,1
1,2,Valparaíso,1
2,3,Concepción,1
3,4,Calama,1
4,5,Buenos Aires,2


Finalmente, modificamos los nombres de las columnas para los dataframe y que estos calcen con el modelo de datos. Luego, procedemos a llenar la tabla **Pais** y luego la tabla **Ciudad**

In [131]:
# Renombramos columnas
cols_pais_mapping = {"idPais": "id"}
cols_ciudad_mapping = {"idCiudad": "id", "nombreCiudad": "NombreCiudad"}

df_pais.rename(columns=cols_pais_mapping, inplace=True)
df_ciudad.rename(columns=cols_ciudad_mapping, inplace=True)

In [132]:
df_pais.head()

,id,nombrePais
0,1,Chile
1,2,Argentina
2,3,Brasil
3,4,Perú
4,5,México


In [133]:
df_ciudad.head()

,id,NombreCiudad,idPais
0,1,Santiago,1
1,2,Valparaíso,1
2,3,Concepción,1
3,4,Calama,1
4,5,Buenos Aires,2


Finalmente, llenamos las tablas almacenados en nuestra base de datos

In [134]:
df_pais.to_sql(
    "Pais",
    con=conn,
    if_exists='append',
    index=False
)

60

In [135]:
df_ciudad.to_sql(
    "Ciudad",
    con=conn,
    if_exists='append',
    index=False
)

82

Finalmente, ejecuta las siguientes celdas para corroborar que los datos se hayan cargado de forma correcta:

In [136]:
%%sql
SELECT * FROM Pais limit 5

 * sqlite:///datos_T3.db
Done.


id,nombrePais
1,Chile
2,Argentina
3,Brasil
4,Perú
5,México


In [137]:
%%sql
SELECT * FROM Ciudad limit 5

 * sqlite:///datos_T3.db
Done.


id,NombreCiudad,idPais
1,Santiago,1
2,Valparaíso,1
3,Concepción,1
4,Calama,1
5,Buenos Aires,2


## Pregunta 4

Llena la tabla LugarNuevo de tu esquema con los datos que se piden. Para esto necesitas ocupar la tabla Lugar (que viene en la base de datos anterior)  y las tablas Ciudad y País (o los CSVs si prefieres). 

Debes tener cuidado al cruzar la información de las ciudades proveniente de la base de datos anterior con las ciudades que vienen de los archivos nuevos, ya que no todas las ciudades tendrán lugares asociados en la base de datos anterior. La idea es que no queden nulos en tu base de datos.


Quizás te puede servir este link https://datatofish.com/sql-to-pandas-dataframe/

### Respuesta

Creamos el Dataframe con los datos de **Lugar**.

In [138]:
curr.execute('''
    SELECT * 
    FROM Lugar
''')

df_lugar = pd.DataFrame(curr.fetchall(), columns=["id", "calle", "numero", "codigo", "nombreCiudad", "nombrePais"])
df_lugar.head()

,id,calle,numero,codigo,nombreCiudad,nombrePais
0,1,"8517 White Circle Suite 464West Christine, WY ...",54408,92721,Glorialand,Syrian Arab Republic
1,2,"0394 Thompson Radial Apt. 440Martinburgh, WI 6...",58837,29150,Jacquelineborough,Saint Vincent and the Grenadines
2,3,"1241 Stevens Extension Suite 769Lopezshire, KS...",23470,49883,Troyberg,Bosnia and Herzegovina
3,4,"3031 Moore FallNorth Amandafort, HI 10818",74384,91003,East Eileenton,Palestinian Territory
4,5,"028 Julia StreamBarbaraburgh, FL 62048",126,75538,Port Katherineport,Marshall Islands


Hacemos un INNER JOIN con el DataFrame *df_ciudad* por medio de las columnas *nombreCiudad* y *NombreCiudad* y *nombrePais*. Utilizamos un INNER JOIN para no dar paso a nulos.

In [139]:
df_ciudad_renamed = df_ciudad.rename(columns={"NombreCiudad": "nombreCiudad", "id": "idCiudad"})
df_pais_renamed = df_ciudad.rename(columns={"id": "idPais"})



df_lugar_ciudad_pais = df_lugar.merge(df_ciudad_renamed, how='inner', on=["nombreCiudad"]).merge(df_pais, how='inner', on=["nombrePais"])
df_lugar_ciudad_pais.head()

,id_x,calle,numero,codigo,nombreCiudad,nombrePais,idCiudad,idPais,id_y
0,1,"8517 White Circle Suite 464West Christine, WY ...",54408,92721,Glorialand,Syrian Arab Republic,23,8,8
1,2,"0394 Thompson Radial Apt. 440Martinburgh, WI 6...",58837,29150,Jacquelineborough,Saint Vincent and the Grenadines,24,9,9
2,58,"8492 Jennifer MillNorth Rayhaven, MI 57576",8480,91557,Port Mary,Saint Vincent and the Grenadines,80,9,9
3,3,"1241 Stevens Extension Suite 769Lopezshire, KS...",23470,49883,Troyberg,Bosnia and Herzegovina,25,10,10
4,4,"3031 Moore FallNorth Amandafort, HI 10818",74384,91003,East Eileenton,Palestinian Territory,26,11,11


Ahora formateamos la pregunta según está definida en nuestro esquema **LugarNuevo**:

In [140]:
cols_lugarnuevo = ["id", "calle", "numero", "codigo", "IDCiudad"]
cols_lugar = ["id_x", "calle", "numero", "codigo", "idCiudad"]

cols = dict(zip(cols_lugar, cols_lugarnuevo))

df_lugar_ciudad_pais = df_lugar_ciudad_pais.rename(columns=cols)
df_lugarnuevo = df_lugar_ciudad_pais[cols_lugarnuevo]

df_lugarnuevo.head()

,id,calle,numero,codigo,IDCiudad
0,1,"8517 White Circle Suite 464West Christine, WY ...",54408,92721,23
1,2,"0394 Thompson Radial Apt. 440Martinburgh, WI 6...",58837,29150,24
2,58,"8492 Jennifer MillNorth Rayhaven, MI 57576",8480,91557,80
3,3,"1241 Stevens Extension Suite 769Lopezshire, KS...",23470,49883,25
4,4,"3031 Moore FallNorth Amandafort, HI 10818",74384,91003,26


Llenamos la tabla **LugarNuevo**:

In [141]:
df_lugarnuevo.to_sql(
    "LugarNuevo",
    con=conn,
    if_exists='append',
    index=False
)

60

Finalmente, ejecuta la siguiente celda para corroborar que los datos se hayan cargado de forma correcta:

In [142]:
%%sql
SELECT * FROM LugarNuevo limit 5

 * sqlite:///datos_T3.db
Done.


id,calle,numero,codigo,IDCiudad
1,"8517 White Circle Suite 464West Christine, WY 25833",54408,92721,23
2,"0394 Thompson Radial Apt. 440Martinburgh, WI 61668",58837,29150,24
3,"1241 Stevens Extension Suite 769Lopezshire, KS 78903",23470,49883,25
4,"3031 Moore FallNorth Amandafort, HI 10818",74384,91003,26
5,"028 Julia StreamBarbaraburgh, FL 62048",126,75538,27


## Pregunta 5

Ahora que terminamos la carga de datos en las tablas nuevas, es tiempo de limpiar nuestra base de datos. Para esto te pedimos borrar las tablas: `Lugar`, `TallerInstrumental` y `TallerDeportivo` de la base de datos antigua.

### Respuesta

Borramos **Lugar**

In [143]:
%%sql

DROP TABLE IF EXISTS Lugar;

 * sqlite:///datos_T3.db
Done.


[]

Borramos **TallerInstrumental**

In [144]:
%%sql

DROP TABLE IF EXISTS TallerInstrumental;

 * sqlite:///datos_T3.db
Done.


[]

Borramos **TallerDeportivo**

In [145]:
%%sql

DROP TABLE IF EXISTS TallerDeportivo;

 * sqlite:///datos_T3.db
Done.


[]

Finalmente, ejecuta las siguientes celdas para corroborar que las tablas se hayan eliminado de forma correcta. Si es que una tabla se borró correctamente, la ejecución de la consulta te debería devolver el siguiente error:
```
* sqlite:///datos_T3.db
(sqlite3.OperationalError) no such table: NombreTabla):
```

In [146]:
%%sql
SELECT * FROM Lugar

 * sqlite:///datos_T3.db
(sqlite3.OperationalError) no such table: Lugar
[SQL: SELECT * FROM Lugar]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [147]:
%%sql
SELECT * FROM TallerDeportivo

 * sqlite:///datos_T3.db
(sqlite3.OperationalError) no such table: TallerDeportivo
[SQL: SELECT * FROM TallerDeportivo]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [148]:
%%sql
SELECT * FROM TallerInstrumental

 * sqlite:///datos_T3.db
(sqlite3.OperationalError) no such table: TallerInstrumental
[SQL: SELECT * FROM TallerInstrumental]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Pregunta 6

Con nuestra base de datos lista, podemos hacer un poco de análisis de los datos. En esta pregunta, te pedimos escribir una consulta SQL que ordene los talleres deportivos según la participación. Quiere decir, debes devolver **id del taller, nombre del taller, nombre del deporte y la cantidad de gente distinta que participa en este taller**, ordenado de forma descendente.

### Respuesta

Para esta consulta, debemos agrupar **Deportivo** segun **Participa**, con un INNER JOIN, descartando aquellos talleres en que no haya participación. Luego debemos hacer un INNER JOIN entre la tabla resultante y **Taller** para obtener el nombre del taller.

In [149]:
%%sql

SELECT Deportivo.id AS idTaller, Taller.nombreTaller, Deportivo.deporte, COUNT(DISTINCT Participa.rut) AS participacion
FROM Deportivo
INNER JOIN Participa
ON Deportivo.id = Participa.idTaller
INNER JOIN Taller
ON Deportivo.id = Taller.id
GROUP BY idTaller
ORDER BY participacion DESC

 * sqlite:///datos_T3.db
Done.


idTaller,nombreTaller,deporte,participacion
76,Taller deportivo 76,Fútbol,9
89,Taller deportivo 89,Fútbol,5
66,Taller deportivo 66,Fútbol,4
92,Taller deportivo 92,Fútbol,3
85,Taller deportivo 85,Voleibol,3
59,Taller deportivo 59,Voleibol,3
37,Taller deportivo 37,Handbol,3
35,Taller deportivo 35,Basquetbol,3
33,Taller deportivo 33,Basquetbol,3
32,Taller deportivo 32,Fútbol,3


## Pregunta 7 

En esta pregunta debes escribir una consulta SQL que entregue el **nombre y país de la ciudad** donde se organiza la máxima cantidad de talleres instrumentales. Quiere decir, hay que calcular cuántos talleres instrumentales hay en cada ciudad, y devolver la(s) ciudad(es) dónde se alcanza el número máximo.


### Respuesta

Para esta consulta debemos agrupar **Ciudad** por la cantidad de talleres instrumentales que hayan en esta. Para esto debemos hacer un INNER JOIN entre **Pais**, **Ciudad**, **LugarNuevo**, **EstaEn** e **Instrumental**. Luego, obtener el maximo de la query.

Primero, creamos una tabla que muestra la información de talleres instrumentales y sus lugares.

In [150]:
%%sql

SELECT *
FROM LugarNuevo
INNER JOIN EstaEn
ON LugarNuevo.id = EstaEn.idLugar
INNER JOIN Instrumental
ON EstaEn.idTaller = Instrumental.id
LIMIT 5


 * sqlite:///datos_T3.db
Done.


id,calle,numero,codigo,IDCiudad,idTaller,idLugar,id_1,instrumento,nrInstrumentos
49,"72174 Meredith Camp Suite 094West Kimberlybury, TX 34755",9579,59956,71,101,49,101,Trompeta,12
27,"7690 Claire Forges Suite 591Davidton, AK 87859",788,49601,49,102,27,102,Bajo,2
43,"53985 Walker SpurJohnhaven, UT 73443",11940,90638,65,103,43,103,Guitarra,4
33,"300 Samantha Trace Apt. 393Port Nicholas, MN 21996",56973,43907,55,104,33,104,Guitarra,4
34,"5528 Gonzales Pike Apt. 637Wallacefort, MN 61797",730,30859,56,105,34,105,Bajo,21


Agrupamos por *LugarNuevo.id*, considerando la cantidad de talleres instrumentales que se realizan en este.

In [151]:
%%sql

SELECT LugarNuevo.id AS idLugarNuevo, COUNT(Instrumental.id) as totalInstrumentales
FROM LugarNuevo
INNER JOIN EstaEn
ON LugarNuevo.id = EstaEn.idLugar
INNER JOIN Instrumental
ON EstaEn.idTaller = Instrumental.id
GROUP BY LugarNuevo.id
ORDER BY totalInstrumentales DESC
LIMIT 10

 * sqlite:///datos_T3.db
Done.


idLugarNuevo,totalInstrumentales
55,4
44,4
32,4
27,4
18,4
16,4
1,4
53,3
48,3
43,3


Como dato, vemos que la mayor cantidad de talleres instrumentales por lugar es 4. Buscamos los ID que tienen la máxima cantidad de talleres instrumentales.

In [152]:
%%sql

SELECT idLugarNuevo, idCiudad
FROM (
    SELECT LugarNuevo.id AS idLugarNuevo, LugarNuevo.IDCiudad AS idCiudad, COUNT(Instrumental.id) as totalInstrumentales
    FROM LugarNuevo
    INNER JOIN EstaEn
    ON LugarNuevo.id = EstaEn.idLugar
    INNER JOIN Instrumental
    ON EstaEn.idTaller = Instrumental.id
    GROUP BY LugarNuevo.id
)
WHERE totalInstrumentales = (
    SELECT MAX(totalInstrumentales)
    FROM (
        SELECT LugarNuevo.id AS idLugarNuevo, COUNT(Instrumental.id) as totalInstrumentales
        FROM LugarNuevo
        INNER JOIN EstaEn
        ON LugarNuevo.id = EstaEn.idLugar
        INNER JOIN Instrumental
        ON EstaEn.idTaller = Instrumental.id
        GROUP BY LugarNuevo.id
    )
)

 * sqlite:///datos_T3.db
Done.


idLugarNuevo,idCiudad
1,23
16,38
18,40
27,49
32,54
44,66
55,77


Finalmente, hacemos un INNER JOIN de esta tabla con las tablas **Ciudad** y **Pais**, para obtener así el nombre del país y de la ciudad donde se organizan más talleres instrumentales.

In [153]:
%%sql


SELECT Ciudad.NombreCiudad, Pais.nombrePais as Pais
FROM Pais
INNER JOIN Ciudad
ON Pais.id = Ciudad.idPais
INNER JOIN (
    SELECT idLugarNuevo, idCiudad
    FROM (
        SELECT LugarNuevo.id AS idLugarNuevo, LugarNuevo.IDCiudad AS idCiudad, COUNT(Instrumental.id) as totalInstrumentales
        FROM LugarNuevo
        INNER JOIN EstaEn
        ON LugarNuevo.id = EstaEn.idLugar
        INNER JOIN Instrumental
        ON EstaEn.idTaller = Instrumental.id
        GROUP BY LugarNuevo.id
        ORDER BY totalInstrumentales DESC
    )
    WHERE totalInstrumentales = (
        SELECT MAX(totalInstrumentales)
        FROM (
            SELECT LugarNuevo.id AS idLugarNuevo, COUNT(Instrumental.id) as totalInstrumentales
            FROM LugarNuevo
            INNER JOIN EstaEn
            ON LugarNuevo.id = EstaEn.idLugar
            INNER JOIN Instrumental
            ON EstaEn.idTaller = Instrumental.id
            GROUP BY LugarNuevo.id
            ORDER BY totalInstrumentales DESC
        )
    )
) AS IDsLugaresConMasInstrumentales
ON Ciudad.id = IDsLugaresConMasInstrumentales.idCiudad


 * sqlite:///datos_T3.db
Done.


NombreCiudad,Pais
Olsonstad,Svalbard & Jan Mayen Islands
North Andrewview,Sweden
West Joseph,Qatar
New Marissamouth,Turkey
Clinemouth,Martinique
South Markborough,Uruguay
Glorialand,Syrian Arab Republic


## Pregunta 8 

En esta pregunta queremos saber los nombres de los países con la cantidad máxima de lugares con al menos 2 talleres deportivos (2 o más) organizados en estos lugares. Quiere decir, para cada lugar hay que ver si hay al menos 2 talleres deportivos en él, y se necesita devolver **el país con la cantidad maxima de lugares** con esta propiedad.

### Respuesta

Primero, agrupamos la cantidad de talleres deportivos por ID de **LugarNuevo**. Similar a la pregunta anterior, también debemos ocupar la tabla **EstaEn**. Filtramos por todos los lugares que tengan 2 o más talleres deportivos.

In [154]:
%%sql

SELECT LugarNuevo.id AS idLugarNuevo, COUNT(Deportivo.id) as totalDeportivos
FROM LugarNuevo
INNER JOIN EstaEn
ON LugarNuevo.id = EstaEn.idLugar
INNER JOIN Deportivo
ON EstaEn.idTaller = Deportivo.id
GROUP BY LugarNuevo.id
HAVING totalDeportivos >= 2
ORDER BY totalDeportivos DESC
LIMIT 50


 * sqlite:///datos_T3.db
Done.


idLugarNuevo,totalDeportivos
33,5
1,5
46,4
43,4
32,4
2,4
57,3
56,3
53,3
44,3


Hacemos un INNER JOIN con **Ciudad** y **Pais** para obtener los ID de los paises de estos lugares.

In [155]:
%%sql

SELECT Pais.nombrePais AS Pais, Ciudad.id AS idCiudad, idLugarNuevo, totalDeportivos
FROM Pais
INNER JOIN Ciudad
ON Pais.id = Ciudad.idPais
INNER JOIN (
    SELECT LugarNuevo.id AS idLugarNuevo, LugarNuevo.idCiudad AS idCiudad, COUNT(Deportivo.id) as totalDeportivos
    FROM LugarNuevo
    INNER JOIN EstaEn
    ON LugarNuevo.id = EstaEn.idLugar
    INNER JOIN Deportivo
    ON EstaEn.idTaller = Deportivo.id
    GROUP BY LugarNuevo.id
    HAVING totalDeportivos >= 2
) AS LugarCon2oMas
ON Ciudad.id = LugarCon2oMas.idCiudad
LIMIT 10


 * sqlite:///datos_T3.db
Done.


Pais,idCiudad,idLugarNuevo,totalDeportivos
Syrian Arab Republic,23,1,5
Saint Vincent and the Grenadines,24,2,4
Palestinian Territory,26,4,2
Israel,28,6,2
United States Minor Outlying Islands,30,8,2
Holy See (Vatican City State),31,9,2
Niger,35,13,2
Estonia,36,14,2
Martinique,40,18,2
Cuba,42,20,2


De la tabla anterior, agrupamos cada país junto al total de sus lugares.

In [156]:
%%sql

SELECT idPais, Pais, COUNT(idLugarNuevo) AS TotalLugares
FROM (
    SELECT Pais.id AS idPais, Pais.nombrePais AS Pais, Ciudad.id AS idCiudad, idLugarNuevo, totalDeportivos
    FROM Pais
    INNER JOIN Ciudad
    ON Pais.id = Ciudad.idPais
    INNER JOIN (
        SELECT LugarNuevo.id AS idLugarNuevo, LugarNuevo.idCiudad AS idCiudad, COUNT(Deportivo.id) as totalDeportivos
        FROM LugarNuevo
        INNER JOIN EstaEn
        ON LugarNuevo.id = EstaEn.idLugar
        INNER JOIN Deportivo
        ON EstaEn.idTaller = Deportivo.id
        GROUP BY LugarNuevo.id
        HAVING totalDeportivos >= 2
    ) AS LugarCon2oMas
    ON Ciudad.id = LugarCon2oMas.idCiudad
)
GROUP BY idPais


 * sqlite:///datos_T3.db
Done.


idPais,Pais,TotalLugares
8,Syrian Arab Republic,1
9,Saint Vincent and the Grenadines,1
11,Palestinian Territory,1
13,Israel,1
15,United States Minor Outlying Islands,1
16,Holy See (Vatican City State),1
19,Niger,1
20,Estonia,1
21,Martinique,1
25,Cuba,1


Finalmente, obtenemos el(los) nombre(s) de los pais(es) con mayor cantidad de lugar que cumplen la condición de tener dos o más talleres deportivos.

In [157]:
%%sql

SELECT Pais
FROM (
    SELECT idPais, Pais, COUNT(idLugarNuevo) AS TotalLugares
    FROM (
        SELECT Pais.id AS idPais, Pais.nombrePais AS Pais, Ciudad.id AS idCiudad, idLugarNuevo, totalDeportivos
        FROM Pais
        INNER JOIN Ciudad
        ON Pais.id = Ciudad.idPais
        INNER JOIN (
            SELECT LugarNuevo.id AS idLugarNuevo, LugarNuevo.idCiudad AS idCiudad, COUNT(Deportivo.id) as totalDeportivos
            FROM LugarNuevo
            INNER JOIN EstaEn
            ON LugarNuevo.id = EstaEn.idLugar
            INNER JOIN Deportivo
            ON EstaEn.idTaller = Deportivo.id
            GROUP BY LugarNuevo.id
            HAVING totalDeportivos >= 2
        ) AS LugarCon2oMas
        ON Ciudad.id = LugarCon2oMas.idCiudad
    )
    GROUP BY idPais
)
WHERE TotalLugares = (
    SELECT MAX(TotalLugares)
    FROM (
        SELECT idPais, Pais, COUNT(idLugarNuevo) AS TotalLugares
        FROM (
            SELECT Pais.id AS idPais, Pais.nombrePais AS Pais, Ciudad.id AS idCiudad, idLugarNuevo, totalDeportivos
            FROM Pais
            INNER JOIN Ciudad
            ON Pais.id = Ciudad.idPais
            INNER JOIN (
                SELECT LugarNuevo.id AS idLugarNuevo, LugarNuevo.idCiudad AS idCiudad, COUNT(Deportivo.id) as totalDeportivos
                FROM LugarNuevo
                INNER JOIN EstaEn
                ON LugarNuevo.id = EstaEn.idLugar
                INNER JOIN Deportivo
                ON EstaEn.idTaller = Deportivo.id
                GROUP BY LugarNuevo.id
                HAVING totalDeportivos >= 2
            ) AS LugarCon2oMas
            ON Ciudad.id = LugarCon2oMas.idCiudad
        )
        GROUP BY idPais
    )
)

 * sqlite:///datos_T3.db
Done.


Pais
Aruba
